# 1. Import de Bibliotecas e de Algoritmos

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import json


# Importar modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

# 2. FEATURE ENGINEERING - Extracao e Limpeza de Dados 

In [2]:
def feature_engeneering(df):
    df_eng = df.copy()

    # --- LIMPEZA INICIAL ---
    # Nota: Não fazemos drop_duplicates no teste para não perder IDs de submissão
    if 'price' in df_eng.columns: 
        df_eng = df_eng.drop_duplicates()

    df_eng['hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False).astype(float)
    df_eng['liters'] = df['engine'].str.extract(r'(\d+\.?\d*)L\s', expand=False).astype(float)

    # --- Idade e Uso ---
    var_ano_atual = date.today().year
    df_eng['car_age'] = var_ano_atual - df_eng['model_year']
    df_eng['car_age'] = df_eng['car_age'].replace(0, 1)

    # --- Cilindrada ---
    df_eng['cylinders'] = df['engine'].str.extract(r'(\d+)\s+Cylinder', expand=False)
    df_eng['cylinders'] = df_eng['cylinders'].fillna(df['engine'].str.extract(r'V(\d+)', expand=False))
    df_eng['cylinders'] = df_eng['cylinders'].astype(float)

    # --- Tecnologias de Motor ---
    df_eng['is_turbo'] = df['engine'].str.contains(r'(?i)turbo', na=False).astype(int)
    df_eng['turbo_type'] = df['engine'].str.extract(r'(Twin Turbo|Turbo)', expand=False)
    df_eng['valve_train'] = df['engine'].str.extract(r'(DOHC|OHV|SOHC)', expand=False) 
    df_eng['fuel_injection'] = df['engine'].str.extract(r'(PDI|GDI|MPFI)', expand=False)

    # Miles per year
    df_eng['miles_p_year'] = df_eng['milage'] / df_eng['car_age']

    # --- FUEL TYPE ---
    def clean_fuel(val):
        s = str(val).lower()
        if 'hybrid' in s: return 'Hybrid'
        elif 'not supported' in s: return 'EV'
        else: return val
    df_eng['fuel_type'] = df_eng['fuel_type'].apply(clean_fuel)

    # --- TRANSMISSION TYPE ---
    def clean_transmission(val):
        s = str(val).lower()
        if 'automatic' in s or 'a/t' in s or 'cvt' in s: return 'Automatico'
        elif 'manual' in s or 'm/t' in s: return 'Manual'
        else: return 'Outro'
    df_eng['transmission_type'] = df_eng['transmission'].apply(clean_transmission)

    # --- Cores (Simplificação) ---
    top_ext_colors = df_eng['ext_col'].value_counts().nlargest(10).index
    df_eng['ext_col_simple'] = df_eng['ext_col'].apply(lambda x: x if x in top_ext_colors else 'Other')

    top_int_colors = df_eng['int_col'].value_counts().nlargest(10).index
    df_eng['int_col_simple'] = df_eng['int_col'].apply(lambda x: x if x in top_int_colors else 'Other')

    # --- Tratamento de Nulos ---
    cols_texto = df_eng.select_dtypes(include=['object']).columns
    df_eng[cols_texto] = df_eng[cols_texto].replace('-', 'Unknown').fillna('Unknown')
    df_eng['clean_title'] = df_eng['clean_title'].replace('Unknown', 'No')

    # --- Acidente ---
    df_eng['accident_clean'] = df_eng['accident'].apply(lambda x: 0 if 'None' in str(x) else 1)

    return df_eng

# 3. PREPARAÇÃO DE DADOS

In [3]:
def preparar_dados(df_treino, df_teste):
    # Aplicar feature engineering
    df_treino_eng = feature_engeneering(df_treino)
    df_teste_eng = feature_engeneering(df_teste)

    # Separar target
    y = df_treino_eng['price']
    X = df_treino_eng.drop('price', axis=1)
    X_test = df_teste_eng.copy()

    features_numericas = ['hp', 'liters', 'car_age', 'cylinders', 'miles_p_year', 
                          'milage', 'model_year', 'is_turbo']

    features_categoricas = ['brand', 'model', 'fuel_type', 'transmission_type', 
                           'ext_col_simple', 'int_col_simple', 'clean_title', 
                           'turbo_type', 'valve_train', 'fuel_injection']

    # --- MELHORIA 1: Preencher numéricos com Mediana em vez de Zero ---
    # Preencher com 0 destrói a lógica (carros com 0 cavalos não existem)
    X_num = X[features_numericas].copy()
    X_test_num = X_test[features_numericas].copy()
    
    for col in features_numericas:
        mediana = X_num[col].median()
        X_num[col] = X_num[col].fillna(mediana)
        X_test_num[col] = X_test_num[col].fillna(mediana)

    # Encodar variáveis categóricas (Label Encoder original)
    X_cat = X[features_categoricas].copy()
    X_test_cat = X_test[features_categoricas].copy()

    encoders = {}
    for col in features_categoricas:
        le = LabelEncoder()
        X_cat[col] = X_cat[col].astype(str)
        le.fit(X_cat[col])
        X_cat[col] = le.transform(X_cat[col])

        # Tratamento para categorias novas no teste
        X_test_cat[col] = X_test_cat[col].astype(str)
        X_test_cat[col] = X_test_cat[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
        encoders[col] = le

    X_final = pd.concat([X_num, X_cat], axis=1)
    X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

    return X_final, y, X_test_final, encoders

# 4. DEFINIÇÃO DE MODELOS

In [4]:
def obter_modelos():
    """Retorna dicionário com todos os modelos disponíveis"""

    modelos = {
        # Regressão Linear
        #'Linear Regression': LinearRegression(),

        # K-Nearest Neighbors
        #'#KNN': KNeighborsRegressor(n_neighbors=5),

        # Árvores de Decisão
        #'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),

        # Random Forest e variantes
        #'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, 
                                               #random_state=42, n_jobs=-1),

        'XGBoost': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, 
                                    max_depth=7, random_state=42, n_jobs=-1),

        # Support Vector Machines
        #'SVR Linear': SVR(kernel='linear', C=1.0),
        #SVR RBF': SVR(kernel='rbf', C=1.0, gamma='scale'),

        # Redes Neuronais
        #'MLP Small': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, 
                                  #random_state=42, early_stopping=True),

        #'MLP Deep': MLPRegressor(hidden_layer_sizes=(100, 50, 25), max_iter=500, 
                                 #random_state=42, early_stopping=True)
    }

    return modelos

# 5. GridSearchCV - Grid Search com Cross Validation

In [5]:
def obter_params_grid_xgb():
    return {
        'n_estimators': [500, 1000, 1500],   # Aumentei um pouco
        'learning_rate': [0.01, 0.05],       # Taxas menores para mais precisão
        'max_depth': [4, 6, 8],
        'subsample': [0.7, 0.8],             # Evita overfitting
        'colsample_bytree': [0.7, 0.8]
    }

def executar_grid_search(X, y):
    print(f"\n🔍 Iniciando GridSearch para XGBoost...")
    
    modelo = xgb.XGBRegressor(random_state=42, n_jobs=-1)
    param_grid = obter_params_grid_xgb()

    grid_search = GridSearchCV(
        estimator=modelo,
        param_grid=param_grid,
        cv=3, 
        scoring='neg_root_mean_squared_error', # Otimiza RMSE
        verbose=1,
        n_jobs=-1
    )

    grid_search.fit(X, y)
    
    print(f"✅ Melhores Parâmetros: {grid_search.best_params_}")
    print(f"   Melhor Score (Log RMSE): {-grid_search.best_score_:,.4f}")
    return grid_search.best_estimator_

# 6. AVALIAÇÃO DE MODELOS

In [6]:
def avaliar_modelo(modelo, X_train, y_train, X_val, y_val, nome_modelo):
    """Treina e avalia um modelo"""

    print(f"\n{'='*60}")
    print(f"Treinando: {nome_modelo}")
    print(f"{'='*60}")

    # Treinar
    modelo.fit(X_train, y_train)

    # Predições
    y_train_pred = modelo.predict(X_train)
    y_val_pred = modelo.predict(X_val)

    # Métricas de treino
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)

    # Métricas de validação
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_r2 = r2_score(y_val, y_val_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)

    # Exibir resultados
    print(f"\n📊 MÉTRICAS DE TREINO:")
    print(f"  RMSE: ${train_rmse:,.2f}")
    print(f"  R²: {train_r2:.4f}")
    print(f"  MAE: ${train_mae:,.2f}")

    print(f"\n📊 MÉTRICAS DE VALIDAÇÃO:")
    print(f"  RMSE: ${val_rmse:,.2f}")
    print(f"  R²: {val_r2:.4f}")
    print(f"  MAE: ${val_mae:,.2f}")

    # Verificar overfitting
    overfit = train_r2 - val_r2
    if overfit > 0.1:
        print(f"\n⚠️  Possível overfitting detectado (diferença R²: {overfit:.4f})")

    return {
        'modelo': nome_modelo,
        'train_rmse': train_rmse,
        'train_r2': train_r2,
        'train_mae': train_mae,
        'val_rmse': val_rmse,
        'val_r2': val_r2,
        'val_mae': val_mae,
        'modelo_treinado': modelo
    }

# 7. TREINAR MODELOS

In [7]:
def treinar_modelos(df_treino, df_teste, modelos_selecionados=None):
    """
    Treina e compara múltiplos modelos

    Args:
        df_treino: DataFrame de treino
        df_teste: DataFrame de teste
        modelos_selecionados: Lista com nomes dos modelos a treinar (None = todos)
    """

    print("🔄 Preparando dados...")
    X, y, X_test, encoders = preparar_dados(df_treino, df_teste)

    # Split treino/validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalizar dados (importante para KNN, SVM e MLP)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)



    print(f"✅ Dados preparados:")
    print(f"  - Treino: {X_train.shape[0]} amostras")
    print(f"  - Validação: {X_val.shape[0]} amostras")
    print(f"  - Features: {X_train.shape[1]}")

    # Obter modelos
    todos_modelos = obter_modelos()

    # Filtrar modelos se especificado
    if modelos_selecionados:
        modelos = {k: v for k, v in todos_modelos.items() if k in modelos_selecionados}
    else:
        modelos = todos_modelos

    # Treinar modelos
    resultados = []
    modelos_treinados = {}

    for nome, modelo in modelos.items():
        # Decidir se usar dados normalizados
        usar_scaled = nome in ['KNN', 'SVR Linear', 'SVR RBF', 'MLP Small', 'MLP Deep']

        if usar_scaled:
            res = avaliar_modelo(modelo, X_train_scaled, y_train, 
                                X_val_scaled, y_val, nome)
        else:
            res = avaliar_modelo(modelo, X_train, y_train, 
                                X_val, y_val, nome)

        resultados.append(res)
        modelos_treinados[nome] = {
            'modelo': res['modelo_treinado'],
            'usar_scaled': usar_scaled
        }

    # Comparação final
    print(f"\n\n{'='*80}")
    print("📈 COMPARAÇÃO DE MODELOS (ordenados por R² de validação)")
    print(f"{'='*80}\n")

    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values('val_r2', ascending=False)

    print(df_resultados[['modelo', 'val_rmse', 'val_r2', 'val_mae']].to_string(index=False))

    # Melhor modelo
    melhor = df_resultados.iloc[0]
    print(f"\n\n🏆 MELHOR MODELO: {melhor['modelo']}")
    print(f"  R² Validação: {melhor['val_r2']:.4f}")
    print(f"  RMSE Validação: ${melhor['val_rmse']:,.2f}")

    return df_resultados, modelos_treinados, scaler, (X_test, X_test_scaled)

# 8. Guardar Hiperparametros 

In [8]:
def salvar_submissao_log(df_sub, modelo_treinado, nome_modelo, metricas):
    """
    Salva CSV e JSON incrementando o ID com base no maior número encontrado.
    """
    pasta = 'submissoes'
    os.makedirs(pasta, exist_ok=True)

    # 1. Listar arquivos e encontrar o maior ID existente
    arquivos = os.listdir(pasta)
    ids_existentes = []

    for f in arquivos:
        # Verifica se o arquivo segue o padrão 'submission_X.csv'
        if f.startswith('submission_') and f.endswith('.csv'):
            try:
                # Extrai apenas o número do nome do arquivo
                # Ex: 'submission_12.csv' -> '12'
                numero_str = f.replace('submission_', '').replace('.csv', '')
                ids_existentes.append(int(numero_str))
            except ValueError:
                continue # Pula arquivos que não tenham número válido

    # Se a lista estiver vazia, começa do 1. Se não, pega o maior + 1
    if not ids_existentes:
        next_id = 1
    else:
        next_id = max(ids_existentes) + 1

    # 2. Definir nomes dos arquivos
    filename_csv = f"{pasta}/submission_{next_id}.csv"
    filename_json = f"{pasta}/submission_{next_id}_params.json"

    # 3. Salvar CSV
    df_sub.to_csv(filename_csv, index=False)

    # 4. Extrair Hiperparâmetros
    try:
        params = modelo_treinado.get_params()
    except:
        params = {"info": "Não foi possível extrair params"}

    # 5. Metadata
    metadata = {
        "id": next_id,
        "modelo": nome_modelo,
        "performance_validacao": metricas,
        "hiperparametros": params
    }

    # 6. Salvar JSON
    with open(filename_json, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=4, default=str)

    print(f"\n✅ Submissão #{next_id} salva com sucesso!")
    print(f"   📂 {filename_csv}")

# Fim - Execucao do Main

In [9]:
if __name__ == "__main__":
    print("📂 Carregando dados...")
    df_treino = pd.read_csv('dados/train.csv', index_col='id')
    df_teste = pd.read_csv('dados/test.csv', index_col='id')

    print("🔄 Preparando dados...")
    X, y, X_test_final, _ = preparar_dados(df_treino, df_teste)

    # --- MELHORIA 2: LOG TRANSFORM OBRIGATÓRIO ---
    # Treinar o GridSearch DIRETAMENTE nos dados LOG
    print("📉 Aplicando Logaritmo no Target para treino...")
    y_log = np.log1p(y)

    # Executar Grid Search (Otimiza os hiperparâmetros para o LOG do preço)
    melhor_modelo = executar_grid_search(X, y_log)

    print(f"\n🏆 Gerando previsões finais...")
    
    # Prever no teste (O resultado sairá em escala Log)
    previsoes_log = melhor_modelo.predict(X_test_final)
    
    # --- MELHORIA 3: INVERSÃO CORRETA ---
    # Converter de volta para Dólares
    previsoes_finais = np.expm1(previsoes_log)

    # Salvar
    df_submissao = pd.DataFrame({
        'id': df_teste.index,
        'price': previsoes_finais
    })

    arquivo_saida = 'submission_xgboost_optimized.csv'
    df_submissao.to_csv(arquivo_saida, index=False)
    print(f"✅ Submissão salva em: {arquivo_saida}")

📂 Carregando dados...
🔄 Preparando dados...
📉 Aplicando Logaritmo no Target para treino...

🔍 Iniciando GridSearch para XGBoost...
Fitting 3 folds for each of 72 candidates, totalling 216 fits
✅ Melhores Parâmetros: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1500, 'subsample': 0.8}
   Melhor Score (Log RMSE): 0.4908

🏆 Gerando previsões finais...
✅ Submissão salva em: submission_xgboost_optimized.csv
